# SmartPlan Notebook
## The study request and Base Cases are available on Box
* Study request: https://smartwires.app.box.com/folder/48021809487
* Base cases: https://smartwires.app.box.com/folder/47866172416
* Contingency files: https://smartwires.app.box.com/file/283783912868 

## The following libraries are required in this notebook:

In [1]:
import os
from bson import ObjectId
import pandas as pd
import numpy as np
from IPython.display import display, HTML, Image
from bson import ObjectId
import Khufu_lib as kl
import chariot as ch


## Working directory

In [ ]:
wd = r'C:\Users\hamid.Elyas\Desktop\SmartPlan'

## Connect to database

In [35]:
server = kl.mongodb_connect('localhost')#IP = ('192.168.15.54')
db = server['ProjectName']

## Submit Investment_Catalog to database
#### conductor_types.csv and sw_devices.csv are two files that should be stored in database 
* Note: Check the ObjectId in collections for both files:
* sw_device.files: ObjectId("59180d46d84e520acc2fba09")
* conductor.files: ObjectId("59180d19d84e520acc2fba07")

In [33]:
# This cell should be run one-time only
server = kl.mongodb_connect('localhost')
db = server['Investment_Catalog']
data = {
       'conductor': [{'filename': 'conductor_types.csv'}],
       'sw_device': [{'filename':'sw_devices.csv' }]
       }
kl.submit_data(db, data, wd)

## Submit Data to database
* commander.csv file contains the list of projects and required files. 
* Using column [include] we have control on the number of projects which should be run.

In [2]:
projects = pd.read_csv(os.path.join(wd, 'commander.csv'))
display(projects)

,index,case,case_index,con,mon,study,invest_option,include,comment
0,1,2020_Min_NAal_GSal_N.raw,1,2020_1.con,2020_1.mon,2020_1_opt,inv_opt_2020_13.csv,N,NaN
1,2,2020_Min_NAal_GSal_PCH_N.raw,2,2020_2.con,2020_2.mon,2020_2_opt,inv_opt_2020_2.csv,N,NaN
2,3,2020_Min_NAal_GSal_PCH_Y.raw,5,2020_5.con,2020_5.mon,NaN,NaN,N,loading less than 100
3,4,2022_Min_NAal_GSal_N.raw,1,2022_1.con,2022_1.mon,2022_1_opt,inv_opt_2022_124.csv,N,NaN
4,5,2022_Min_NAal_GSal_PCH_N.raw,2,2022_2.con,2022_2.mon,2022_2_opt,inv_opt_2022_124.csv,N,NaN
5,6,2022_Max_NAba_GSal_Y.raw,3,2022_3.con,2022_3.mon,2022_3_opt,inv_opt_2022_3.csv,N,NaN
6,7,2025_Min_NAal_GSal_N.raw,1,2025_1.con,2025_1.mon,2025_1_opt,inv_opt_2025_12.csv,N,NaN
7,8,2025_Min_NAal_GSal_PCH_N.raw,2,2025_1.con,2025_1.mon,2025_2_opt,inv_opt_2025_12.csv,Y,NaN
8,9,2025_Max_NAba_GSal_Y.raw,3,2025_3.con,2025_3.mon,2025_3_opt,inv_opt_2025_3.csv,N,NaN
9,10,2030_Min_NAal_GSal_N.raw,1,2030_1.con,2030_1.mon,2030_1_opt,inv_opt_2025_12.csv,N,NaN


In [37]:
for x in range(len(projects)):
    if projects['include'][x] == 'Y':
        data = {
            'case': [{'filename': projects['case'][x]}],
            'con': [{'filename': projects['con'][x]}],
            'mon': [{'filename': projects['mon'][x]}],
            'invOpt': [{'filename': projects['invest_option'][x]}]
            }
        kl.submit_data(db, data, wd)

## N-1 Replication

In [ ]:
s = projects[projects.include == 'Y']

for i in range(len(s)):
    if not s.run.iloc[i]:
        continue
    db.studies.delete_one({'name' : s.name.iloc[i]})
    study = {
        'study': {
            'name': s.name.iloc[i],
            'status': 'incomplete',
            'type': 'n1'
        },
        'settings': {
            "tara_version": "1702",
            "loading_cutoff": "90",
            "contRating" : "2"
        },
        'criteria': {
            'case': {
                'filename': s.case.iloc[i]
            },
            'con': {
                'filename': s.con.iloc[i]
            }
        }
    }
    if type(s.inch.iloc[i]) == str:
        study['criteria'].update({'inch':{'filename': s.inch.iloc[i]}})
        
    kl.build_study(db, study)

## Filter the obtained results

In [ ]:
kl.get_study_results(db, wd)
s = studies[studies.type == 'n1']

for i in range(len(s)):
    name = s.name.iloc[i]
    if not s.run.iloc[i]:
        continue
    print name
    
    ss = db.studies.find_one({'name' :name})
    if ss is None:
        continue
    if ss['status'] != 'complete':
        continue
        
    # Read in raw results
    n1 = pd.read_csv(os.path.join(wd,  name, 'n1_thermal.csv'))
    # Filter to the lines we care about
    n1['ACLoad'] = pd.to_numeric(n1['ACLoad'], errors='coerce')
    n1 = n1[['FrmName', 'FrmBus', 'ToName', 'ToBus', 'Ckt', 'kVs', 'Ctg', 'CtgRate', 'ACLoad']]
    # Display the worst loading scenario for each facility 
    display(HTML('<h2>{}<h2>'.format(name)))
    display(n1[n1.ACLoad > 100].head())

## Optimization

In [38]:
db.studies.delete_many({})

for x in range(len(projects)):
    if studies['include'][x] == 'Y':
        opt_study = {
                    'study':{
                            'name': studies['study'][x],
                            'status': 'incomplete',
                            'type': 'optimization'
                            },
                    'settings':{
                                "tara_version": "1702",
                                "secure_to_percent": "99",
                                "loading_cutoff": "30",
                                "gbrt_kappa": "1.96",
                                "gbrt_n_random_starts": "15",
                                "gbrt_n_calls": "100",
                                "ctgPenalty": "250000"
                                },
                    'criteria':{
                                'case': {'filename': projects['case'][x]},
                                'con': {'filename': projects['con'][x]},
                                'mon': {'filename': projects['mon'][x]},
                                'invOpt': {'filename': projects['invest_option'][x]}
                                }
                     }

        kl.build_study(db, opt_study)

In [20]:
# Download the results
kl.get_study_results(db, wd)
# The optained results can be filtered 